In [ ]:
import os
import re
import pandas as pd

pattern = r"function\s+(\w+)\s*\((.*?)\)\s*(public|external|internal|private)?\s*(view|pure|override)?"
df = pd.DataFrame(columns=['Contract Name', 'Read Functions', 'Write Functions'])
for file_name in os.listdir("meme/"):
    if file_name.endswith(".sol"):
        with open(os.path.join("meme/", file_name), "r", encoding="utf-8") as f:
            code = f.read()
            matches = re.findall(pattern, code)

            read_functions = []
            write_functions = []
            for match in matches:
                func_name = match[0]
                attributes = match[2:]
                if "view" in attributes or "pure" in attributes:
                    read_functions.append(func_name)
                else:
                    write_functions.append(func_name)
            contract_name = file_name.replace('.sol', '')
            temp_df = pd.DataFrame([{
                'Contract Name': contract_name,
                'Read Functions': ', '.join(read_functions),
                'Write Functions': ', '.join(write_functions)
            }])

            df = pd.concat([df, temp_df], ignore_index=True)

df.to_csv("contract_functions.csv", index=False, encoding='utf-8-sig')


In [ ]:
import pandas as pd
from collections import Counter

df = pd.read_csv("contract_functions.csv")
all_read_functions = []
all_write_functions = []

total_rows = len(df)

for _, row in df.iterrows():
    if isinstance(row["Read Functions"], str):
        unique_read_functions = set(row["Read Functions"].split(", "))
        all_read_functions.extend(unique_read_functions)
    
    if isinstance(row["Write Functions"], str):
        unique_write_functions = set(row["Write Functions"].split(", "))
        all_write_functions.extend(unique_write_functions)

read_function_counts = Counter(all_read_functions)
write_function_counts = Counter(all_write_functions)

read_df = pd.DataFrame([(func, count, (count / total_rows) * 100) for func, count in read_function_counts.items()],
                       columns=["Read Function", "Count", "Percentage (%)"])
write_df = pd.DataFrame([(func, count, (count / total_rows) * 100) for func, count in write_function_counts.items()],
                        columns=["Write Function", "Count", "Percentage (%)"])

with pd.ExcelWriter("function_percentage_counts.xlsx") as writer:
    read_df.sort_values(by="Percentage (%)", ascending=False).to_excel(writer, sheet_name="Read Functions", index=False)
    write_df.sort_values(by="Percentage (%)", ascending=False).to_excel(writer, sheet_name="Write Functions", index=False)
